In [ ]:
# pip install bambi

In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#   Modelos lineales y modelos lineales generalizados
# ----------------------------------------------------------
#        Rolando Gonzales Martinez, Agosto 2024
# ==========================================================
#             MLGs Bayesianos wald y gamma
# ==========================================================
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import warnings
az.style.use("arviz-darkgrid")
np.random.seed(666)
warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
# --------------------- Cargar datos ----------------------------------
url = "https://raw.githubusercontent.com/rogon666/UMSA/main/MLMLG/datos/UFV_IPC.csv"
datos = pd.read_csv(url)
print(datos.head())

In [ ]:
# Agregar una nueva columna de fecha comenzando en enero de 2006, con frecuencia mensual
datos['fecha'] = pd.date_range(start='2006-01-01', periods=len(datos), freq='M')

# Cambio IPC a 12 meses
datos['IPC_cambio_12m'] = datos['IPC'].pct_change(periods=12) * 100

# Raiz 12th de 'IPC_cambio_12m'
datos['IPC_raiz_12'] = np.power(datos['IPC_cambio_12m'], 1/12)
print(datos.head(24))

# Raiz 30th de 'IPC_raiz_12', proxy inflacion diaria
datos['inflacion_diaria'] = np.power(datos['IPC_raiz_12'], 1/30)
print(datos.tail(24))

datos['UFV_proxy'] = datos['inflacion_diaria'] * datos['UFV'].shift(1)

# Calcular los cambios porcentuales mensuales para UFV y UFV proxy
datos['cambio_pct_UFV'] = datos['UFV'].pct_change() * 100
datos['cambio_pct_UFV_proxy'] = datos['UFV_proxy'].pct_change() * 100

In [ ]:
# Guardando datos de 2023
UFVs_holdout = datos['UFV'].tail(17)
inflacion_diaria_holdout = datos['inflacion_diaria'].tail(17)
datos = datos.dropna()
datos = datos.iloc[:-17] # Removiendo los ultimos 17 datos
print(datos.tail())

In [ ]:
# Graficar la serie temporal
plt.figure(figsize=(14, 8))

# Serie temporal para los cambios porcentuales de UFV
plt.subplot(3, 1, 1)
plt.plot(datos['fecha'], datos['cambio_pct_UFV'], marker='o', linestyle='-', color='skyblue')
plt.title('Serie Temporal de los cambios de las UFVs')
plt.xlabel('Fecha')
plt.ylabel('Cambio Porcentual (%)')
plt.grid(True)

# Serie temporal para los cambios porcentuales de UFV proxy
plt.subplot(3, 1, 2)
plt.plot(datos['fecha'], datos['cambio_pct_UFV_proxy'], marker='o', linestyle='-', color='skyblue')
plt.title('Serie Temporal de los cambios de las UFVs proxy')
plt.xlabel('Fecha')
plt.ylabel('Cambio Porcentual (%)')
plt.grid(True)

# Serie temporal
plt.subplot(3, 1, 3)
plt.plot(datos['fecha'], datos['inflacion_diaria'], marker='o', linestyle='-', color='lightgreen')
plt.title('Serie Temporal de la inflacion diaria')
plt.xlabel('Fecha')
plt.ylabel('Cambio Porcentual (%)')
plt.grid(True)
plt.show()

In [ ]:
# Histogramas
datos = datos[datos["cambio_pct_UFV"] > 0]
plt.figure(figsize=(14, 6))

# Histograma para los cambios porcentuales del UFV
plt.subplot(1, 2, 1)
plt.hist(datos['cambio_pct_UFV'], bins=20, color='skyblue', edgecolor='black')
plt.title('Cambios Porcentuales Mensuales de la UFV')
plt.xlabel('Cambio Porcentual (%)')
plt.ylabel('Frecuencia')

# Histograma para los cambios porcentuales del IPC
plt.subplot(1, 2, 2)
plt.hist(datos['inflacion_diaria'], bins=20, color='lightgreen', edgecolor='black')
plt.title('Cambios Porcentuales Mensuales de la inflacion diaria')
plt.xlabel('Cambio Porcentual (%)')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Diagrama cruzado
import seaborn as sns
plt.figure(figsize=(10, 6))
sns.regplot(x='cambio_pct_UFV', y='inflacion_diaria', data=datos, ci=95)
plt.title('Correlacion entre las variaciones de la UFV y la inflacion diaria')
plt.xlabel('variaciones UFV')
plt.ylabel('inflacion diaria')
plt.grid(True)
plt.show()

In [ ]:
# MLG Bayesiano de Wald
modelo_wald = bmb.Model("cambio_pct_UFV ~ C(Gobierno) + inflacion_diaria", 
                       datos, family = "wald", link = "log")
ajuste_wald = modelo_wald.fit(draws = 100,
                              chains = 2,
                             init="jitter+adapt_diag")
az.plot_trace(ajuste_wald)
az.summary(ajuste_wald)

In [ ]:
# MLG Bayesiano Gamma
modelo_gamma = bmb.Model(
    "cambio_pct_UFV ~ Gobierno + inflacion_diaria",
    datos,
    family="gamma",
    link="log",
    categorical="Gobierno"
)
ajuste_gamma = modelo_gamma.fit(draws = 100,
                                chains = 2,
                               target_accept=0.9, 
                               init="jitter+adapt_diag")
az.plot_trace(ajuste_gamma)
az.summary(ajuste_gamma)

In [ ]:
# Predicciones
Lucho = ["Lucho", "Lucho", "Lucho", "Lucho", "Lucho", "Lucho",
         "Lucho", "Lucho", "Lucho", "Lucho", "Lucho", "Lucho",
         "Lucho", "Lucho", "Lucho", "Lucho", "Lucho", "Lucho",
         "Lucho", "Lucho"]
Evo = ["Evo", "Evo", "Evo", "Evo", "Evo", "Evo",
       "Evo", "Evo", "Evo", "Evo", "Evo", "Evo",
       "Evo", "Evo", "Evo", "Evo", "Evo", "Evo",
       "Evo", "Evo"]
predicciones_gamma = bmb.interpret.predictions(
    modelo_gamma, 
    ajuste_gamma,  
    conditional={
        "inflacion_diaria": np.array(inflacion_diaria_holdout),
        "Gobierno": Lucho
        }
)
print(predicciones_gamma)
predicciones_wald = bmb.interpret.predictions(
    modelo_wald, 
    ajuste_wald,  
    conditional={
        "inflacion_diaria": np.array(inflacion_diaria_holdout),
        "Gobierno": Lucho
        }
)
print(predicciones_wald)
df = pd.DataFrame(predicciones_gamma['inflacion_diaria'])
df['pred_gamma'] = pd.DataFrame(predicciones_gamma['estimate'])
df['pred_wald'] = pd.DataFrame(predicciones_wald['estimate'])
print(df)

In [ ]:
# UFV Diciembre 2022
initial_ufv = 2.40898

df['UFV_proyectada_gamma'] = initial_ufv
df['UFV_proyectada_wald'] = initial_ufv
for i in range(1, len(df)):
    df.loc[i, 'UFV_proyectada_gamma'] = df.loc[i-1, 'UFV_proyectada_gamma'] * (1 + df.loc[i, 'pred_gamma'] / 100)
    df.loc[i, 'UFV_proyectada_wald'] = df.loc[i-1, 'UFV_proyectada_wald'] * (1 + df.loc[i, 'pred_wald'] / 100)
    
print(df)

In [ ]:
# RMSE
from sklearn.metrics import mean_squared_error
# Calcular el RMSE entre UFV observada y UFV proyectada
rmse_gamma = np.sqrt(mean_squared_error(UFVs_holdout, df['UFV_proyectada_gamma']))
rmse_wald  = np.sqrt(mean_squared_error(UFVs_holdout, df['UFV_proyectada_wald']))
# Mostrar el RMSE
print(f"RMSE gamma: {rmse_gamma}")
print(f"RMSE  wald: {rmse_wald}")

In [ ]:
# Graficos en la muestra hold out
# Generar las fechas correspondientes
fechas = pd.date_range(start='2023-01-01', periods=len(UFVs_holdout), freq='M')

# Ejemplo de datos (ajusta a tus datos reales)
datag = {
    'Fecha': fechas,
    'UFV observada': UFVs_holdout.reset_index(drop=True),
    'UFV proyectada gamma': df['UFV_proyectada_gamma'],
    'UFV proyectada wald': df['UFV_proyectada_wald']
}

dfg = pd.DataFrame(datag)

# Gráfico de la serie de tiempo con área de confianza
plt.figure(figsize=(10, 5))
plt.plot(dfg['Fecha'], dfg['UFV observada'], marker='o', label='UF Observada')
plt.plot(dfg['Fecha'], dfg['UFV proyectada gamma'], marker='o', label='UFV Proyectada gamma')
plt.plot(dfg['Fecha'], dfg['UFV proyectada wald'], marker='o', label='UFV Proyectada wald')

plt.title('Comparación de UFV Observada vs UFV Proyectada')
plt.xlabel('Fecha')
plt.ylabel('Valor de UFV')
plt.legend()
plt.grid(True)
plt.show()